In [1]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

/Users/yongqifan/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open('../data/UMLS/2017AB/2017AB/META/MRCONSO.RRF','r') as f:
    L = f.readlines()

In [3]:
con_2017AB = [i.strip() for i in L]

In [5]:
meddra_2017 = [i for i in con_2017AB if i.split('|')[1]=='ENG' and 'MDR' in i.split('|')[11] and i.split('|')[12]=='PT']

In [7]:
meddra_pt_2017 = [i.split('|')[13]+'##'+i.split('|')[14] for i in con_2017AB if i.split('|')[1]=='ENG' and 'MDR'== i.split('|')[11] and i.split('|')[12]=='PT']

In [9]:
con_2017AB = [i.strip().split('|') for i in tqdm(L) if '|ENG|' in i]

In [11]:
meddra_pt_2017_name =  [i[14] for i in con_2017AB if i[1]=='ENG' and 'MDR'== i[11] and i[12]=='PT']

In [12]:
meddra_pt_2017_code =  [i[13] for i in con_2017AB if i[1]=='ENG' and 'MDR'== i[11] and i[12]=='PT']


In [13]:
len(meddra_pt_2017_code)

22499

In [14]:
meddra_dict = {}
for i in meddra_pt_2017:
    item = i.split('##')
    meddra_dict[item[0]] = item[1]

In [ ]:
import json
with open('../data/SMM4H-17/smm4h_normalized_set.json','r',encoding='utf-8') as f:
    smm4h_normalized_set = json.load(f)

In [17]:
code2ptcode = {}

code2ptcode['10014698'] = '10014695'
code2ptcode['10007541'] = '10061024'
code2ptcode['10012259'] = '10012239'
code2ptcode['10000497'] = '10000496'
code2ptcode['10040991'] = '10040984'
code2ptcode['10044027'] = '10012318'

In [ ]:
smm4h_code = []

for num in range(4):
    with open('../data/SMM4H-17/task_3_normalization_training'+str(num+1)+'.txt') as f:
        L = f.readlines()
        for i in L:
            item = i.strip().split('\t')
            if item[-1] in code2ptcode:
                smm4h_code.append(code2ptcode[item[-1]])
            else:
                smm4h_code.append(item[-1])

with open('../data/SMM4H-17/task_3_normalization_evaluation.txt') as f:
    L = f.readlines()
    for i in L:
        item = i.strip().split('\t')
        if item[-1] in code2ptcode:
            smm4h_code.append(code2ptcode[item[-1]])
        else:
            smm4h_code.append(item[-1])

smm4h_code= list(set(smm4h_code))

In [19]:
len(smm4h_code)

509

In [20]:
len([i for i in smm4h_code if i not in meddra_pt_2017_code])

4

In [21]:
oov_code = [i for i in smm4h_code if i not in meddra_pt_2017_code]
len(oov_code)

4

In [22]:
oov_code

['10013663', '10024130', 'MEDDRA PT', '10027433']

In [25]:
['|'.join(i) for i in con_2017AB if i[13] in oov_code and i[1]=='ENG']

['C0023222|ENG|S|L0023222|PF|S0675616|Y|A25743967|||10033425|MDR|LLT|10024130|Leg pain|3|N|256|',
 'C0851358|ENG|P|L1378196|PF|S1635896|N|A1595997|||10027433|MDR|OS|10027433|Metabolism and nutrition disorders|3|N||',
 'C0851358|ENG|S|L6331855|PF|S8789416|Y|A14224797|||10027433|MDR|AB|10027433|Metab|3|Y||',
 'C1510472|ENG|P|L0013170|VC|S0368302|N|A27876856|||10079381|MDR|LLT|10013663|Drug dependence|3|N|256|']

In [26]:
meddra_dict['10024130'] = 'Leg pain'
meddra_dict['10013663'] = 'Drug dependence'
meddra_dict['10027433'] = 'Metabolism and nutrition disorders'
meddra_dict['MEDDRA PT'] = 'MEDDRA PT'

In [28]:
norm_set = list(set([meddra_dict[i] for i in list(meddra_dict)]))
norm_set.sort()

In [29]:
len(norm_set)

22503

In [ ]:
import json
with open('../data/SMM4H-17/smm4h_normalized_set.json','w') as f:
    json.dump(norm_set,f,ensure_ascii=False,indent=4)

In [ ]:
with open('../data/SMM4H-17/smm4h_code2ptcode.json','w') as f:
    json.dump(code2ptcode,f,ensure_ascii=False,indent=4)

In [ ]:
with open('../data/SMM4H-17/smm4h_code2term.json','w') as f:
    json.dump(meddra_dict,f,ensure_ascii=False,indent=4)

In [35]:
name2aid = {}
aid2name = {}
for i in tqdm(con_2017AB):
    if i[14] in norm_set:
        k = i[14]
        v = i[7]
        name2aid[k] = v
        aid2name[v] = k   

100%|██████████| 9176008/9176008 [16:04<00:00, 9514.37it/s] 


In [ ]:
with open('../data/UMLS/2023AB/META/MRDEF.RRF','r') as f:
    L1 = f.readlines()

In [37]:
denf = [i.strip().split('|') for i in L1]

In [38]:
denf.sort()

In [39]:
aid_set = list(aid2name)

In [45]:
denf[0]

['C0000039',
 'A0016515',
 'AT38152019',
 '',
 'MSH',
 'Synthetic phospholipid used in liposomes and lipid bilayers to study biological membranes. It is also a major constituent of PULMONARY SURFACTANTS.',
 'N',
 '',
 '']

In [46]:
term2denf = {}
for i in tqdm(denf):
    if i[1] in aid_set:
        term2denf[aid2name[i[1]]]=i[5]

100%|██████████| 430839/430839 [09:33<00:00, 750.72it/s]


In [47]:
len(term2denf)

2030

In [ ]:
with open('../data/SMM4H-17/smm4h_umls_kc.json','w') as f:
    json.dump(term2denf,f,ensure_ascii=False,indent=4)